In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../title_maker_pro")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import re
import stanza
import pickle
import datasets
from collections import Counter
import dictionary_definition
stanza.download('en')  
from transformers import AutoModelWithLMHead, AutoTokenizer

2020-05-04 00:51:11 INFO: Downloading default packages for language: en (English)...
2020-05-04 00:51:12 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-05-04 00:51:15 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [6]:
blacklist = datasets.Blacklist(set())
blacklist.merge(datasets.Blacklist.from_text_stream(open("/mnt/evo/projects/title-maker-pro/data/wikitext-103-raw/wiki.train.raw"), use_gpu=True))
blacklist.merge(datasets.Blacklist.from_text_stream(open("/mnt/evo/projects/title-maker-pro/data/wikitext-103-raw/wiki.test.raw"),  use_gpu=True))
blacklist.merge(datasets.Blacklist.from_text_lines(open("/mnt/evo/projects/title-maker-pro/data/words.txt")))
blacklist.merge(datasets.Blacklist.from_parsed_dictionary("/mnt/evo/projects/title-maker-pro/data/en_dictionary_parsed_randomized.pickle"))

2020-05-04 00:52:38 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-04 00:52:38 INFO: Use device: gpu
2020-05-04 00:52:38 INFO: Loading: tokenize
2020-05-04 00:52:39 INFO: Done loading processors!
2020-05-04 00:53:04 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-04 00:53:04 INFO: Use device: gpu
2020-05-04 00:53:04 INFO: Loading: tokenize
2020-05-04 00:53:04 INFO: Done loading processors!
2020-05-04 00:53:28 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-05-04 00:53:28 INFO: Use device: gpu
2020-05-04 00:53:28 INFO: Loading: tokenize
2020-05-04 00:53:28 INFO: Done loading processors!
2020-05-04 00:53:54 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     

In [8]:
len(blacklist.blacklist_set)

5763399

In [ ]:
blacklist.merge(datasets.Blacklist.from_text_lines(open("../data/words.txt")))

In [7]:
blacklist.dump("../build/blacklist.pickle")

In [ ]:
parsed_dict = pickle.load(open("../data/en_dictionary_parsed_randomized.pickle", "rb"))

In [ ]:
has_period = 0
no_period = 0
c = Counter()
for item in parsed_dict[:1000]:
    for sense in item.senses:
        for definition in sense.definitions:
            if isinstance(definition, dictionary_definition.ReferenceDefinition):
                continue
                
            c[definition.definition.strip()[-1]] += 1
            if definition.definition.strip().endswith("."):
                has_period += 1
            else:
                no_period += 1
                
print(f"Has period: {100 * has_period / (has_period + no_period):.2f}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
ip = datasets.InverseParsedDictionaryDefinitionDataset(tokenizer, None, None)

In [ ]:
for item in parsed_dict[:100]:
    for i, sense in enumerate(item.senses):
        for definition in sense.definitions:
            if isinstance(definition, dictionary_definition.ReferenceDefinition):
                continue
            
            if definition.definition.strip()[-1] == ".":
                e = ip._make_examples(tokenizer, item)[i]
                if e:
                    print(tokenizer.decode(e))

In [ ]:
_)